In [5]:
from tensorflow.keras.layers import Input, Conv2D, SeparableConv2D, \
    Add, Dense ,BatchNormalization, ReLU, MaxPool2D, GlobalAvgPool2D

In [24]:
def conv_bn(x,filters,kernel_size,strides=1):
    x=Conv2D(filters=filters,kernel_size=kernel_size,strides=strides, padding='same',use_bias=False)(x)
    x=BatchNormalization()(x)
    return x

In [25]:
def sep_bn(x,filters,kernel_size,strides=1):
    x=SeparableConv2D(filters=filters,kernel_size=kernel_size,strides=strides, padding='same',use_bias=False)(x)
    x=BatchNormalization()(x)
    return x

In [18]:
def entry_flow(x):
    x=conv_bn(x,filters=32, kernel_size=3, strides=2)
    x=ReLU()(x)
    x=conv_bn(x,filters=64, kernel_size=3)
    tensor=ReLU()(x)    
    x=sep_bn(tensor, filters=128,kernel_size=3)
    x=ReLU()(x)
    x=sep_bn(x, filters=128,kernel_size=3)
    x=MaxPool2D(pool_size=3,strides=2,padding='same')(x)    
    tensor=conv_bn(tensor, filters=128, kernel_size=1,strides=2)
    x=Add()([tensor,x])    
    x=ReLU()(x)
    x=sep_bn(x, filters=256,kernel_size=3)
    x=ReLU()(x)
    x=sep_bn(x, filters=256,kernel_size=3)
    x=MaxPool2D(pool_size=3,strides=2,padding='same')(x)
    tensor=conv_bn(tensor, filters=256, kernel_size=1,strides=2)
    x=Add()([tensor,x])    
    x=ReLU()(x)
    x=sep_bn(x, filters=728,kernel_size=3)
    x=ReLU()(x)
    x=sep_bn(x, filters=728,kernel_size=3)
    x=MaxPool2D(pool_size=3,strides=2,padding='same')(x)    
    tensor=conv_bn(tensor, filters=728, kernel_size=1,strides=2)
    x=Add()([tensor,x])    
    return x

    

    
    

In [19]:
def middle_flow(tensor):
    for _ in range(8):
        x=ReLU()(tensor)
        x=sep_bn(x, filters=728,kernel_size=3)
        x=ReLU()(x)
        x=sep_bn(x, filters=728,kernel_size=3)
        x=ReLU()(x)
        x=sep_bn(x, filters=728,kernel_size=3)
        tensor=Add()([tensor,x])
    return tensor    
    

In [27]:
def exit_flow(tensor):
    x=ReLU()(tensor)
    x=sep_bn(x, filters=728,kernel_size=3)
    x=ReLU()(x)
    x=sep_bn(x, filters=1024,kernel_size=3)
    x=MaxPool2D(pool_size=3,strides=2,padding='same')(x)
    tensor=conv_bn(tensor, filters=1024, kernel_size=1,strides=2)
    x=Add()([tensor,x])
    x=sep_bn(x, filters=1536,kernel_size=3)
    x=ReLU()(x)
    x=sep_bn(x, filters=2048,kernel_size=3)
    x=ReLU()(x)
    x=GlobalAvgPool2D()(x)
    x=Dense(units=1000,activation='softmax')(x)    
    return x



In [28]:
input=Input(shape=(299,299,3))
x=entry_flow(input)
x=middle_flow(x)
output=exit_flow(x)

from tensorflow.keras import Model
model=Model(inputs=input,outputs=output)

In [42]:
from tensorflow.keras.utils import plot_model
import pydotplus
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True,
    rankdir='TB')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
